## 1. Загрузка и предобработка данных

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats as st

In [ ]:
tt = pd.read_csv('../input/task-time/task_about_time.tsv', sep='\t')
tt.head(3)

Преобразование timestamp в datetime и вычисление трудоёмкости (длительности) задания

In [ ]:
tt['start_ts'] = pd.to_datetime(tt['start_ts'],unit='s')
tt['end_ts'] = pd.to_datetime(tt['end_ts'],unit='s')
tt['dur'] = tt['end_ts'] - tt['start_ts']
tt['dur'] = tt['dur'].dt.seconds

## 2. EDA

Формирование таблицы характеристик набора данных в разрезе категориальных признаков

In [ ]:
pr1 = tt[tt['project_id'] == 1]['dur'].describe().to_frame()
pr1.columns = ['project_1']
pr2 = tt[tt['project_id'] == 2]['dur'].describe().to_frame()
pr2.columns = ['project_2']

res_1 = tt[tt['resolution_type'] == 'resolution_1']['dur'].describe().to_frame()
res_1.columns = ['resolution_1']
res_2 = tt[tt['resolution_type'] == 'resolution_2']['dur'].describe().to_frame()
res_2.columns = ['resolution_2']

tot = tt['dur'].describe().to_frame()
tot.columns = ['total']

des = pr1.join(pr2)\
         .join(res_1)\
         .join(res_2)\
         .join(tot)

des

Удалений заданий с резолюцией 2, как имеющих признаки выбросов. Их доля в 8,5% позволит считать оставшуюся часть данных репрезентативной.

In [ ]:
tt = tt[tt['resolution_type'] != 'resolution_2']

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (20,5))

tt[tt['project_id'] == 1]['dur'].hist(bins = 100, ax=ax[0])
tt[tt['project_id'] == 2]['dur'].hist(bins = 100, ax=ax[0])
ax[0].set_title('Распределение длительности в разрезе заданий')
ax[0].legend(('project_1', 'project_2'))
ax[0].set_xlabel('сек.')
ax[0].set_ylabel('количество')

tt.groupby('worker')['dur'].count().plot.bar()

ax[1].set_title('Количество выполненных заданий по асессорам')
ax[1].set_ylabel('количество')

plt.show()

Данные положительно скошены и длительность проектов распределена одинаково. При этом можно выделить TOP-3 ассесоров по количеству выполненных заданий - login16, login7, login9.

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (20,5))

tt[tt['project_id'] == 1][['worker', 'dur']].boxplot(by='worker', rot=45, ax=ax[0])
tt[tt['project_id'] == 2][['worker', 'dur']].boxplot(by='worker', rot=45, ax=ax[1])

ax[0].set_ylabel('сек.')
ax[0].set_title('project_1')
ax[1].set_title('project_2')
fig.suptitle("Диграммы размаха длительности по ассесорам")

plt.show()

In [ ]:
data1 = tt[~tt['worker'].isin(['login15','login9'])]['dur']
data2 = tt[tt['worker'].isin(['login15','login9'])]['dur']

stat, p = st.mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
	print('Выборки статистически равны')
else:
	print('Нельзя утверждать, что выборки статистически равны')

Оценка распределения длительности у каждого ассесора позволяет выделить две группы асессоров. В первой - меньшее количество выполненных заданий и присутствие выбросов, у второй, login15 и login9, - большее количество выполненных заданий без аномальных значений. Эти группы являются статистически разными.

Лучшие качественные характеристики позволяют предложить в качестве бенчмарка длительности среднюю длительность второй группы. Дополнительно стоит отметить, что средняя предлагается в противовес  медиане, несмотря на значительную положительную скошенность датасета. В данной случае, на примере лидеров очевидно, что присутствует значительное количество заданий, которые требуют большей трудоёмкости, чем в среднем, соответственно, более высокая оплата будет являться справедливым и стимулирующим фактором для выполения большего объёма заданий.

## 3. Расчёт бенчмарка длительности

In [ ]:
print('Медиана длительности для TOP-ассесоров: {} сек.'
      .format(tt[tt['worker'].isin(['login15','login9'])]['dur']
              .mean()))

## 4. Выводы

В качестве нормируемой трудоёмкости предлагается 2003 секунды и, при условии оплаты N за 30 секунд, оплату 67N за задание.